## Network Traffic Dataset for malicious attack

This dataset of network traffic flow is generated by CICFlowMeter, indicate whether the traffic is malicious attack (Bot) or not (Benign).                             
CICFlowMeter - network traffic flow generator generates 69 statistical features such as Duration, Number of packets, Number of bytes, Length of packets, etc are also calculated separately in the forward and reverse direction.   
The output of the application is the CSV file format with two columns labeled for each flow, namely Benign or Bot.
The dataset has been organized per day, for each day the raw data including the network traffic (Pcaps) and event logs (windows and Ubuntu event Logs) per machine
are recorded.                  Download the dataset from the below wget command line provided.

In [1]:
! wget -O Network_Traffic.csv https://cse-cic-ids2018.s3.ca-central-1.amazonaws.com/Processed+Traffic+Data+for+ML+Algorithms/Friday-02-03-2018_TrafficForML_CICFlowMeter.csv

--2020-06-30 05:24:15--  https://cse-cic-ids2018.s3.ca-central-1.amazonaws.com/Processed+Traffic+Data+for+ML+Algorithms/Friday-02-03-2018_TrafficForML_CICFlowMeter.csv
Resolving cse-cic-ids2018.s3.ca-central-1.amazonaws.com (cse-cic-ids2018.s3.ca-central-1.amazonaws.com)... 52.95.145.136
Connecting to cse-cic-ids2018.s3.ca-central-1.amazonaws.com (cse-cic-ids2018.s3.ca-central-1.amazonaws.com)|52.95.145.136|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 352368373 (336M) [text/csv]
Saving to: ‘Friday-02-03-2018_TrafficForML_CICFlowMeter.csv’

Friday-02-03-2018_T 100%[===================>] 336.04M  21.5MB/s    in 12s     

2020-06-30 05:24:28 (27.5 MB/s) - ‘Friday-02-03-2018_TrafficForML_CICFlowMeter.csv’ saved [352368373/352368373]



## Install Libraries

In [1]:
! pip install pandas --user
! pip install imblearn --user

You should consider upgrading via the 'pip install --upgrade pip' command.
You should consider upgrading via the 'pip install --upgrade pip' command.


## Restart Notebook Kernel

In [ ]:
from IPython.display import display_html
display_html("<script>Jupyter.notebook.kernel.restart()</script>",raw=True)

## Import Libraries

In [1]:
import tensorflow as tf
import pandas as pd
import numpy as np
import tempfile
import os
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder,MinMaxScaler
from sklearn.model_selection import KFold
from imblearn.combine import SMOTETomek
from imblearn.over_sampling import RandomOverSampler
from sklearn.feature_selection import SelectKBest
from sklearn.feature_selection import f_classif

from kubernetes.client import V1Container
from kubernetes.client import V1ResourceRequirements
from kfserving import KFServingClient
from kfserving import constants
from kfserving import utils
from kfserving import V1alpha2EndpointSpec
from kfserving import V1alpha2PredictorSpec
from kfserving import V1alpha2InferenceServiceSpec
from kfserving import V1alpha2InferenceService
from kfserving import V1alpha2CustomSpec
from kfserving import V1alpha2TensorflowSpec

from kubernetes import client as k8s_client
from kubernetes import config as k8s_config
from kubernetes.client.rest import ApiException

import tensorflow
import requests, json
import numpy as np

k8s_config.load_incluster_config()
api_client = k8s_client.CoreV1Api()
custom_api=k8s_client.CustomObjectsApi()
namespace = utils.get_default_target_namespace()
print(namespace)

anonymous


### Check number of available GPUs

In [2]:
print("Num GPUs Available: ", len(tf.config.experimental.list_physical_devices('GPU')))
tf.config.experimental.list_physical_devices('GPU')

Num GPUs Available:  2


[PhysicalDevice(name='/physical_device:GPU:0', device_type='GPU'),
 PhysicalDevice(name='/physical_device:GPU:1', device_type='GPU')]

### Declare Variables

In [3]:
lstZerodrp = ['Timestamp', 'BwdPSHFlags', 'FwdURGFlags', 'BwdURGFlags', 'CWEFlagCount', 'FwdBytsbAvg', 'FwdPktsbAvg',
              'FwdBlkRateAvg', 'BwdBytsbAvg',
              'BwdBlkRateAvg', 'BwdPktsbAvg']

lstScaledrp = ['FwdPSHFlags', 'FINFlagCnt', 'SYNFlagCnt', 'RSTFlagCnt', 'PSHFlagCnt', 'ACKFlagCnt', 'URGFlagCnt',
               'ECEFlagCnt']

DATA_FILE = 'Network_Traffic.csv'

In [4]:
def read_dataFile():
    """
    Reads data file and returns dataframe result
    """
    chunksize = 100000
    chunk_list = []
    missing_values = ["n/a", "na", "--", "Infinity", "infinity", "Nan", "NaN"]

    for chunk in pd.read_csv(DATA_FILE, chunksize=chunksize, na_values=missing_values):
        chunk_list.append(chunk)
#         break
    dataFrme = pd.concat(chunk_list)

    lstcols = []
    for i in dataFrme.columns:
        i = str(i).replace(' ', '').replace('/', '')
        lstcols.append(i)
    dataFrme.columns = lstcols
    dfAllCpy = dataFrme.copy()
    dataFrme = dataFrme.drop(lstZerodrp, axis=1)
    return dataFrme

## Network Traffic Input Dataset 

### Attribute Information
    Features extracted from the captured traffic using CICFlowMeter-V3 = 69
    After removal of noise/unwarranted features, number of feature columns chosen: 10
    Features: FlowDuration,BwdPktLenMax,FlowIATStd,FwdPSHFlags,BwdPktLenMean,FlowIATMean,BwdIATMean,
              FwdSegSizeMin,InitBwdWinByts,BwdPktLenMin
    Flows labelled: Bot or Benign

In [5]:
read_dataFile().head()

,DstPort,Protocol,FlowDuration,TotFwdPkts,TotBwdPkts,TotLenFwdPkts,TotLenBwdPkts,FwdPktLenMax,FwdPktLenMin,FwdPktLenMean,...,FwdSegSizeMin,ActiveMean,ActiveStd,ActiveMax,ActiveMin,IdleMean,IdleStd,IdleMax,IdleMin,Label
0,443,6,141385,9,7,553,3773.0,202,0,61.444444,...,20,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,Benign
1,49684,6,281,2,1,38,0.0,38,0,19.000000,...,20,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,Benign
2,443,6,279824,11,15,1086,10527.0,385,0,98.727273,...,20,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,Benign
3,443,6,132,2,0,0,0.0,0,0,0.000000,...,20,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,Benign
4,443,6,274016,9,13,1285,6141.0,517,0,142.777778,...,20,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,Benign


In [6]:
def preprocess_na(dataFrme):
    """
    Removing NA values
    """
    na_lst = dataFrme.columns[dataFrme.isna().any()].tolist()
    for j in na_lst:
        dataFrme[j].fillna(0, inplace=True)
    return dataFrme

In [7]:
def create_features_label(dataFrme):
    """
    Create independent and Dependent Features
    """
    columns = dataFrme.columns.tolist()
    # Filter the columns to remove data we do not want 
    columns = [c for c in columns if c not in ["Label"]]
    # Store the variable we are predicting 
    target = "Label"
    # Define a random state 
    state = np.random.RandomState(42)
    X = dataFrme[columns]
    Y = dataFrme[target]
    return X, Y

In [8]:
def label_substitution(dataFrme):
    """
    Label substitution : 'Benign'as 0, 'Bot'as 1
    """
    dictLabel = {'Benign': 0, 'Bot': 1}
    dataFrme['Label'] = dataFrme['Label'].map(dictLabel)

    LABELS = ['Benign', 'Bot']
    count_classes = pd.value_counts(dataFrme['Label'], sort=True)
    
    # Get the Benign and the Bot values 
    Benign = dataFrme[dataFrme['Label'] == 0]
    Bot = dataFrme[dataFrme['Label'] == 1]
    return dataFrme

In [9]:
def handle_class_imbalance(X,Y):
    """
    Handle Class imbalancement 
    """
#    os_us = SMOTETomek(ratio=0.5)
#    X_res, y_res = os_us.fit_sample(X, Y)
    ros = RandomOverSampler(random_state=50)
    X_res, y_res = ros.fit_sample(X, Y)
    ibtrain_X = pd.DataFrame(X_res,columns=X.columns)
    ibtrain_y = pd.DataFrame(y_res,columns=['Label']) 
    return ibtrain_X,ibtrain_y

In [10]:
def correlation_features(ibtrain_X):
    """
    Feature Selection - Correlation Ananlysis 
    """
    corr = ibtrain_X.corr()
    cor_columns = np.full((corr.shape[0],), True, dtype=bool)
    for i in range(corr.shape[0]):
        for j in range(i + 1, corr.shape[0]):
            if corr.iloc[i, j] >= 0.9:
                if cor_columns[j]:
                    cor_columns[j] = False

    dfcorr_features = ibtrain_X[corr.columns[cor_columns]]
    return dfcorr_features

In [11]:
def top_ten_features(dfcorr_features,ibtrain_X,ibtrain_y):
    feat_X = dfcorr_features
    feat_y = ibtrain_y['Label']
    
    #apply SelectKBest class to extract top 10 best features
    bestfeatures = SelectKBest(score_func=f_classif, k=10)
    fit = bestfeatures.fit(feat_X,feat_y)
    dfscores = pd.DataFrame(fit.scores_)
    dfcolumns = pd.DataFrame(feat_X.columns)
    #concat two dataframes for better visualization 
    featureScores = pd.concat([dfcolumns,dfscores],axis=1)
    featureScores.columns = ['Features','Score']  #naming the dataframe columns
    final_feature = featureScores.nlargest(10,'Score')['Features'].tolist()
    final_feature.sort()
    sort_fn = final_feature
    dictLabel1 = {'Benign':0,'Bot':1}
    ibtrain_y['Label']= ibtrain_y['Label'].map(dictLabel1)
    selected_X = ibtrain_X[sort_fn]
    selected_Y = ibtrain_y['Label']
    return selected_X,selected_Y,sort_fn

In [12]:
def normalize_data(selected_X, selected_Y):
    """
    Normalize data 
    """
    scaler = MinMaxScaler(feature_range=(0, 1))
    selected_X = pd.DataFrame(scaler.fit_transform(selected_X), columns=selected_X.columns, index=selected_X.index)
    trainX, testX, trainY, testY = train_test_split(selected_X, selected_Y, test_size=0.25)
    print('-----------------------------------------------------------------')
    print("## Final features and Data pre-process for prediction")
    print('-----------------------------------------------------------------')
    print(testX)
    return trainX, testX, trainY, testY

In [13]:
tf.logging.set_verbosity(tf.logging.INFO)
    
'''Reads data file and returns dataframe result'''
dataFrme = read_dataFile()

''' Removing NA values'''
dataFrme = preprocess_na(dataFrme)

'''Create independent and Dependent Features'''
X, Y = create_features_label(dataFrme)

'''Label substitution : 'Benign'as 0, 'Bot'as 1'''
dataFrme = label_substitution(dataFrme)

'''Handle Class imbalancement'''
ibtrain_X, ibtrain_y = handle_class_imbalance(X, Y)

'''Feature Selection - Correlation Ananlysis'''
dfcorr_features = correlation_features(ibtrain_X)

'''Feature Selection - SelectKBest : Return best 10 features'''
selected_X, selected_Y, final_feature = top_ten_features(dfcorr_features, ibtrain_X, ibtrain_y)

'''Normalize data '''
trainX, testX, trainY, testY = normalize_data(selected_X, selected_Y)

-----------------------------------------------------------------
## Final features and Data pre-process for prediction
-----------------------------------------------------------------
         BwdPktLenMax  BwdPktLenMean  BwdPktLenMin  FlowDuration  FlowIATMax  \
1028334      0.000000       0.000000      0.000000      0.716435    0.716667   
162436       0.803425       0.154737      0.000000      0.016137    0.007944   
929926       1.000000       0.326308      0.000000      0.042618    0.041102   
1450785      0.000000       0.000000      0.000000      0.000004    0.000004   
1017053      0.000000       0.000000      0.000000      0.001540    0.001539   
...               ...            ...           ...           ...         ...   
1152205      0.000000       0.000000      0.000000      0.000005    0.000005   
751402       0.034932       0.034941      0.035664      0.000006    0.000006   
339282       0.076712       0.022095      0.000000      0.000094    0.000087   
1138411      0

## Definition of Serving Input Receiver Function

In [14]:
def make_feature_cols():
  input_columns = [tf.feature_column.numeric_column(k,dtype=tf.dtypes.float64) for k in final_feature]
  return input_columns
feature_columns =  make_feature_cols()
inputs = {}
for feat in feature_columns:
  inputs[feat.name] = tf.placeholder(shape=[None], dtype=feat.dtype)
serving_input_receiver_fn = tf.estimator.export.build_raw_serving_input_receiver_fn(inputs)

## Train and Save Network Traffic Model

In [15]:
TF_DATA_DIR = os.getenv("TF_DATA_DIR", "/tmp/data/")
TF_MODEL_DIR = os.getenv("TF_MODEL_DIR", "network/")
TF_EXPORT_DIR = os.getenv("TF_EXPORT_DIR", "network/")

x1 = np.asarray(trainX[final_feature])
y1 = np.asarray(trainY)

x2 = np.asarray(testX[final_feature])
y2 = np.asarray(testY)

In [16]:
def formatFeatures(features):
    formattedFeatures = {}
    numColumns = features.shape[1]

    for i in range(0, numColumns):
        formattedFeatures[final_feature[i]] = features[:, i]

    return formattedFeatures

In [17]:
config = tf.estimator.RunConfig(model_dir=TF_MODEL_DIR, save_summary_steps=10, save_checkpoints_steps=10)

# Provide list of GPUs should be used to train the model
trainingFeatures = formatFeatures(x1)
trainingCategories = y1

testFeatures = formatFeatures(x2)
testCategories = y2


distribution = tf.distribute.experimental.ParameterServerStrategy()
print('Number of devices: {}'.format(distribution.num_replicas_in_sync))

# Configuration of  training model
config = tf.estimator.RunConfig(train_distribute=distribution, model_dir=TF_MODEL_DIR, save_summary_steps=100, save_checkpoints_steps=1000)

model = tf.estimator.DNNClassifier(hidden_units=[13,65,110],
                                   feature_columns=feature_columns,
                                   model_dir=TF_MODEL_DIR,
                                   n_classes=2, config=config
                                   )
# Train Input Function
def train_input_fn():
    dataset = tf.data.Dataset.from_tensor_slices((trainingFeatures, y1))
    dataset = dataset.batch(128).repeat(10000)
    return dataset

# Test Input Function
def eval_input_fn():
    dataset = tf.data.Dataset.from_tensor_slices((testFeatures, y2))
    return dataset.batch(128).repeat(10000)

export_final = tf.estimator.FinalExporter(TF_EXPORT_DIR, serving_input_receiver_fn=serving_input_receiver_fn)

train_spec = tf.estimator.TrainSpec(input_fn=train_input_fn,
                                    max_steps=10000)

eval_spec = tf.estimator.EvalSpec(input_fn=eval_input_fn,
                                  steps=100,
                                  exporters=export_final,
                                  throttle_secs=1,
                                  start_delay_secs=1)

result = tf.estimator.train_and_evaluate(model, train_spec, eval_spec)
print(result)

print('Training finished successfully')

INFO:tensorflow:ParameterServerStrategy with compute_devices = ('/device:GPU:0', '/device:GPU:1'), variable_device = '/device:CPU:0'


[I 200701 05:18:10 parameter_server_strategy:294] ParameterServerStrategy with compute_devices = ('/device:GPU:0', '/device:GPU:1'), variable_device = '/device:CPU:0'


Number of devices: 2
INFO:tensorflow:Initializing RunConfig with distribution strategies.


[I 200701 05:18:10 run_config:566] Initializing RunConfig with distribution strategies.


INFO:tensorflow:Not using Distribute Coordinator.


...

INFO:tensorflow:SavedModel written to: network/export/network/temp-b'1593580793'/saved_model.pb


[I 200701 05:19:55 builder_impl:425] SavedModel written to: network/export/network/temp-b'1593580793'/saved_model.pb


INFO:tensorflow:Loss for final step: 46.398563.


[I 200701 05:19:55 estimator:371] Loss for final step: 46.398563.


({'accuracy': 0.9267188, 'accuracy_baseline': 0.5007031, 'auc': 0.96505505, 'auc_precision_recall': 0.9688145, 'average_loss': 0.17515133, 'label/mean': 0.49929687, 'loss': 22.41937, 'precision': 0.8726254, 'prediction/mean': 0.49645966, 'recall': 0.99906117, 'global_step': 10000}, [b'network/export/network/1593580793'])
Training finished successfully


## KFServing Client SDK
We will use the KFServing client SDK to create the InferenceService and deploy our custom image.

### Define InferenceService
Firstly define default endpoint spec, and then define the inferenceservice using the endpoint spec.

To use a custom image we need to use V1alphaCustomSpec which takes a V1Container from the kuberenetes library

In [18]:
MODEL_NAME="network-model"
pvcname = !(echo  $HOSTNAME | sed 's/.\{2\}$//')
pvc = "workspace-"+pvcname[0]
api_version = constants.KFSERVING_GROUP + '/' + constants.KFSERVING_VERSION
default_endpoint_spec = V1alpha2EndpointSpec(
                          predictor=V1alpha2PredictorSpec(
                              tensorflow=V1alpha2TensorflowSpec(
                              storage_uri='pvc://%s/network/export/network'%pvc,
                              resources=V1ResourceRequirements(limits={"nvidia.com/gpu": 1})
                              )))

isvc = V1alpha2InferenceService(api_version=api_version,
                          kind=constants.KFSERVING_KIND,
                          metadata=k8s_client.V1ObjectMeta(
                              name=MODEL_NAME, namespace=namespace),
                                spec=V1alpha2InferenceServiceSpec(default=default_endpoint_spec)
                                )

## Create the InferenceService
Call KFServingClient to create InferenceService.

In [19]:
KFServing = KFServingClient()
KFServing.create(isvc)

{'apiVersion': 'serving.kubeflow.org/v1alpha2',
 'kind': 'InferenceService',
 'metadata': {'creationTimestamp': '2020-07-01T05:21:26Z',
  'generation': 1,
  'name': 'network-model',
  'namespace': 'anonymous',
  'resourceVersion': '3258062',
  'selfLink': '/apis/serving.kubeflow.org/v1alpha2/namespaces/anonymous/inferenceservices/network-model',
  'uid': 'a6d24fbe-e013-4ed6-84b2-eb331999c819'},
 'spec': {'default': {'predictor': {'tensorflow': {'resources': {'limits': {'cpu': '1',
       'memory': '2Gi',
       'nvidia.com/gpu': '1'},
      'requests': {'cpu': '1', 'memory': '2Gi'}},
     'runtimeVersion': '1.14.0-gpu',
     'storageUri': 'pvc://workspace-ravi/network/export/network'}}}},
 'status': {}}

## Check the InferenceService

In [20]:
KFServing.get(MODEL_NAME, namespace=namespace, watch=True)

NAME                 READY      DEFAULT_TRAFFIC CANARY_TRAFFIC  URL                                               
network-model        False                                                                                        
network-model        False                                                                                        
network-model        False                                                                                        
network-model        False                                                                                        
network-model        True       100                             http://network-model.anonymous.example.com/v1/m...


## Run a prediction

In [21]:
CLUSTER='ucs' #where your cluster running 'gcp' or 'ucs'

In [22]:
%%bash -s "$CLUSTER" --out NODE_IP
if [ $1 = "ucs" ]
then
    echo "$(kubectl get node -o=jsonpath='{.items[0].status.addresses[0].address}')"
else
    echo "$(kubectl get node -o=jsonpath='{.items[0].status.addresses[1].address}')"
fi

In [23]:
%%bash --out INGRESS_PORT
INGRESS_GATEWAY="istio-ingressgateway"
echo "$(kubectl -n istio-system get service $INGRESS_GATEWAY -o jsonpath='{.spec.ports[1].nodePort}')"

In [24]:
%%bash -s "$MODEL_NAME" --out SERVICE_HOSTNAME
echo "$(kubectl get inferenceservice $1 -o jsonpath='{.status.url}' | cut -d "/" -f 3)"

In [26]:
url = f"http://{NODE_IP.strip()}:{INGRESS_PORT.strip()}/v1/models/{MODEL_NAME}:predict"
headers = {"Host": SERVICE_HOSTNAME.strip()}
data={"signature_name":"predict","instances":[{"BwdPktLenMax":[0.158904] , "BwdPktLenMean":[0.039736] , "BwdPktLenMin":[0.00000], "FlowDuration":[0.053778] , "FlowIATMax":[0.053262] , "FwdPktLenMin":[0.0] , "FwdSegSizeMin":[0.454545] , "InitBwdWinByts":[1.0] , "Protocol":[0.0] , "RSTFlagCnt":[0.003357]}]}
response=requests.post(url, data=json.dumps(data), headers=headers)
print(response.json())

{'predictions': [{'logits': [-205.913147], 'class_ids': [0], 'classes': ['0'], 'all_class_ids': [0, 1], 'logistic': [0.0], 'all_classes': ['0', '1'], 'probabilities': [1.0, 0.0]}]}


## Clean Up

## Delete the InferenceService

In [27]:
KFServing.delete(MODEL_NAME, namespace=namespace)

{'kind': 'Status',
 'apiVersion': 'v1',
 'metadata': {},
 'status': 'Success',
 'details': {'name': 'network-model',
  'group': 'serving.kubeflow.org',
  'kind': 'inferenceservices',
  'uid': 'a6d24fbe-e013-4ed6-84b2-eb331999c819'}}

In [28]:
!rm -rf network